In [162]:
import os
import pandas as pd
import numpy as np
import pickle
import ast
from scipy import sparse
import logging

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import  ElasticNetCV
from sklearn.svm import SVC

In [163]:
trn = pd.read_csv('train_luc.csv', sep=',', header=None)
print(len(trn))
trn.head(2)

1600


,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
0,976,0,0,0,0.000000,0,0,0.004348,0,0,...,0.02381,1,0,0,1,0,1,1,0,1
1,2530,0,0,0,0.026316,0,0,0.000000,0,0,...,0.00000,1,0,0,1,0,0,0,0,1


In [164]:
la = MultiLabelBinarizer()
la.fit_transform([[0,1,2,3,4,5,6,7,8],[2,4,6,7,8]])
la.classes_

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [165]:
features = []
labels = []
test_feats = []

In [166]:
for r in range(len(trn)):
    tmpfeat = []
    for c in range(1,246):
        tmpfeat.append(trn[c][r])
    features.append(tmpfeat)

trn['features'] = features

In [167]:
for r in range(len(trn)):
    tmplabels = []
    for c in range(246,255):
        tmplabels.append(trn[c][r])
    labels.append(tmplabels)

trn['labels'] = labels

In [168]:
trn.head(2)

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,features,labels
0,976,0,0,0,0.000000,0,0,0.004348,0,0,...,0,0,1,0,1,1,0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0043478260869...","[1, 0, 0, 1, 0, 1, 1, 0, 1]"
1,2530,0,0,0,0.026316,0,0,0.000000,0,0,...,0,0,1,0,0,0,0,1,"[0.0, 0.0, 0.0, 0.0263157894737, 0.0, 0.0, 0.0...","[1, 0, 0, 1, 0, 0, 0, 0, 1]"


In [169]:
trn.to_csv('trn.csv', index=None)

In [170]:
classif = OneVsRestClassifier(ElasticNetCV(max_iter=10000), n_jobs=-2)

In [174]:
np_labels = np.array(labels)

In [175]:
classif.fit(features,np_labels)

OneVsRestClassifier(estimator=ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=10000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0),
          n_jobs=-2)

In [176]:
test = pd.read_csv('test.csv', sep=',', header=None)
print(len(test))
test.head(2)

10000


,0,1,2,3,4,5,6,7,8,9,...,236,237,238,239,240,241,242,243,244,245
0,0fenh,0.000000,0,0.007874,0.000000,0.012821,0.000000,0.000000,0.000000,0.00000,...,0,0,0,0.013889,0.011364,0.021277,0.015385,0.000000,0.008333,0.021739
1,iw8p1,0.003155,0,0.003175,0.007092,0.013514,0.003817,0.005405,0.014493,0.00339,...,0,0,0,0.011236,0.000000,0.006410,0.111111,0.005155,0.006329,0.090909


In [177]:
for r in range(len(test)):
    tmpfeat = []
    for c in range(1,246):
        tmpfeat.append(test[c][r])
    test_feats.append(tmpfeat)

In [178]:
len(test_feats)

10000

In [179]:
test['features'] = test_feats

In [180]:
test.head(2)

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,features
0,0fenh,0.000000,0,0.007874,0.000000,0.012821,0.000000,0.000000,0.000000,0.00000,...,0,0,0.013889,0.011364,0.021277,0.015385,0.000000,0.008333,0.021739,"[0.0, 0.0, 0.00787401574803, 0.0, 0.0128205128..."
1,iw8p1,0.003155,0,0.003175,0.007092,0.013514,0.003817,0.005405,0.014493,0.00339,...,0,0,0.011236,0.000000,0.006410,0.111111,0.005155,0.006329,0.090909,"[0.00315457413249, 0.0, 0.0031746031746, 0.007..."


In [181]:
test.to_csv('test_mean.csv', index=None)

In [182]:
labelarray = classif.predict(test_feats)

In [185]:
len(labelarray)

10000

In [189]:
labels = la.inverse_transform(labelarray)
labels[:5]

[(1, 2, 5, 6),
 (1, 2, 4, 5, 6),
 (0, 2, 3, 5, 6, 8),
 (1, 2, 3, 5, 6, 8),
 (1, 2, 5, 6, 8)]

In [190]:
test['labels'] = labels

In [191]:
test.head(2)

,0,1,2,3,4,5,6,7,8,9,...,238,239,240,241,242,243,244,245,features,labels
0,0fenh,0.000000,0,0.007874,0.000000,0.012821,0.000000,0.000000,0.000000,0.00000,...,0,0.013889,0.011364,0.021277,0.015385,0.000000,0.008333,0.021739,"[0.0, 0.0, 0.00787401574803, 0.0, 0.0128205128...","(1, 2, 5, 6)"
1,iw8p1,0.003155,0,0.003175,0.007092,0.013514,0.003817,0.005405,0.014493,0.00339,...,0,0.011236,0.000000,0.006410,0.111111,0.005155,0.006329,0.090909,"[0.00315457413249, 0.0, 0.0031746031746, 0.007...","(1, 2, 4, 5, 6)"


In [203]:
sub = pd.DataFrame({"business_id":test[0].values, "labels":test['labels'].values})

In [204]:
for i,t in enumerate(sub['labels'].values): 
    sub['labels'].values[i] = ' '.join(map(str,t))

In [205]:
sub.head(2)

,business_id,labels
0,0fenh,1 2 5 6
1,iw8p1,1 2 4 5 6


In [206]:
len(sub)

10000

In [207]:
sub.to_csv('mebasar_submission2.csv', index=None)